In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('all')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
data = pd.read_csv("data.csv")
data.head()

,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,discounted_price,specifications,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,NaN,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8gb', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,https://www.mega.pk/mobiles_products/24393/Tec...,Tecno Spark 10,['https://www.mega.pk/items_images/Tecno+Spark...,Tecno,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '4GB,8GB', 'Memory quantity': '', 'Int...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
3,3,https://www.mega.pk/mobiles_products/24259/Viv...,Vivo V27 5G,['https://www.mega.pk/items_images/Vivo+V27+5G...,Vivo,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB,12GB', 'Memory quantity': '', 'In...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
4,4,https://www.mega.pk/mobiles_products/24204/App...,Apple Iphone 15 Pro Max,['https://www.mega.pk/items_images/Apple+Iphon...,Apple,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [ ]:
data.describe()

,id,used,original_price,discounted_price,delivery_fee,delivery_details,average_rating,num_ratings
count,1666.00000,1666.00000,1.649000e+03,739.000000,107.0,0.0,900.000000,900.000000
mean,832.50000,0.02521,1.615732e+05,41247.326116,0.0,NaN,4.992222,13.331111
std,481.07709,0.15681,2.470286e+05,96631.352491,0.0,NaN,0.096925,43.200123
min,0.00000,0.00000,1.199000e+03,1149.000000,0.0,NaN,3.500000,1.000000
25%,416.25000,0.00000,6.000000e+03,3899.500000,0.0,NaN,5.000000,1.000000
50%,832.50000,0.00000,5.199900e+04,6599.000000,0.0,NaN,5.000000,1.000000
75%,1248.75000,0.00000,2.420000e+05,29299.000000,0.0,NaN,5.000000,6.000000
max,1665.00000,1.00000,4.457999e+06,794999.000000,0.0,NaN,5.000000,751.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1666 non-null   int64  
 1   slug              1666 non-null   object 
 2   title             1666 non-null   object 
 3   imgs              1666 non-null   object 
 4   brand             773 non-null    object 
 5   category          1666 non-null   object 
 6   vendor            1666 non-null   object 
 7   used              1666 non-null   int64  
 8   address           544 non-null    object 
 9   availability      1079 non-null   object 
 10  currency          1666 non-null   object 
 11  original_price    1649 non-null   float64
 12  discounted_price  739 non-null    float64
 13  specifications    1666 non-null   object 
 14  description       299 non-null    object 
 15  delivery_fee      107 non-null    float64
 16  delivery_details  0 non-null      float64


In [ ]:
# # Importing pandas library
# import pandas as pd
# import numpy as np

# # Creating a sample DataFrame with NaN values
# sales_data = pd.DataFrame({
#   'Product': ['A', 'B', 'C', 'D', 'E'],
#   'Price': [10.5, 8.0, np.nan, 15.5, np.nan],
#   'Units_Sold': [100, 150, np.nan, 80, 200],
#   'In_Stock': [True, False, np.nan, True, np.nan]
# })

# # Display the original DataFrame
# print("Original Sales Data:")
# print(sales_data)

# # Creating a dictionary with column-specific fill values
# fill_values = {
#   'Price': 0.0,
#   'Units_Sold': 0,
#   'In_Stock': False
# }

# # Filling NaN values with column-specific values
# filled_sales = sales_data.fillna(fill_values)

# # Display the result
# print("\nSales Data after filling NaN values:")
# print(filled_sales)


In [ ]:
data.isnull().sum()

,0
id,0
slug,0
title,0
imgs,0
brand,893
category,0
vendor,0
used,0
address,1122
availability,587


In [ ]:
filled_value = {
    'brand': 'False',
    'address': 'False',
    'availability': 'False',
    'original_price': 0.0,
    'discounted_price': 0.0,
    'description': 'False',
    'delivery_fee': 0.0,
    'delivery_details': 0.0,
    'warranty': 'False',
    'warranty_type': 'False',
    'average_rating': 0.0,
    'num_ratings': 0.0,





}

In [ ]:
datas = data.fillna(filled_value)

In [ ]:
datas.isnull().sum()

,0
id,0
slug,0
title,0
imgs,0
brand,0
category,0
vendor,0
used,0
address,0
availability,0


In [ ]:
# from surprise import SVD, Dataset, Reader, accuracy
# from surprise.model_selection import train_test_split

# # Create a sample dataset
# ratings = {
#     'user': ['A', 'A', 'B', 'B', 'C', 'C'],
#     'item': ['Item1', 'Item2', 'Item1', 'Item3', 'Item2', 'Item3'],
#     'rating': [1, 2, 2, 3, 3, 1]
# }

# df = pd.DataFrame(ratings)

# # Define a reader and the scale of ratings
# reader = Reader(rating_scale=(1, 3))

# # Load the dataset from the DataFrame
# data = Dataset.load_from_df(df, reader)

# # Split data into training and test set
# trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# # Train an SVD model
# model = SVD()
# model.fit(trainset)

# # Predict ratings for the test set
# predictions = model.test(testset)

# # Calculate RMSE for the predictions
# rmse = accuracy.rmse(predictions)
# print(f"Test Set RMSE: {rmse:.3f}")

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import linear_kernel

# # Sample data
# data = {'movie': ['Movie1', 'Movie2', 'Movie3'],
#         'description': ['Action thriller', 'Romantic comedy', 'Action romance']}
# df = pd.DataFrame(data)

# # Compute the TF-IDF matrix
# tfidf = TfidfVectorizer(stop_words='english')
# df['description'] = df['description'].fillna('')
# tfidf_matrix = tfidf.fit_transform(df['description'])

# # Compute cosine similarity
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# # Get movie recommendations based on cosine similarity
# def get_recommendations(title, cosine_sim=cosine_sim):
#     idx = df.index[df['movie'] == title].tolist()[0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:3]
#     movie_indices = [i[0] for i in sim_scores]
#     return df['movie'].iloc[movie_indices]

# print(get_recommendations('Movie1'))

In [ ]:
porter_stem = PorterStemmer()

def tokenization(text):
  res= text.lower()
  res= re.sub(r'/d+', '', text)
  res= re.sub(r'/W', '', text)
  res= re.sub(r'/w/s', '', text)
  res= re.sub(r'^a-zA-Z0-9', '', text)
  res= re.sub(r'\s+', ' ', text).strip()
  res= [porter_stem.stem(word) for word in res if not word in stopwords.words('english')]
  res = ''.join(res)

  return res

In [ ]:
datas['description'][2]

'False'

In [ ]:
datas.head(2)

,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,discounted_price,specifications,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,False,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",False,...,0.0,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",False,0.0,0.0,False,False,0.0,0.0,[]
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",False,...,0.0,"{'RAM': '8gb', 'Memory quantity': '', 'Interna...",False,0.0,0.0,False,False,0.0,0.0,[]


In [ ]:

# tfidf_matrix = tfidf.fit_transform(datas['title'])

In [ ]:
def cosine_similarity_matrix(text1, text2):
    # Initialize the TF-IDF Vectorizer for english words
    vectorizer = TfidfVectorizer(tokenizer=tokenization)

    # Fit and transform the new text feature to create the TF-IDF matrix
    text_vectors = vectorizer.fit_transform([text1, text2])

    # Calculate the cosine similarity matrix between all 'texts'
    cosine_sim_matrix = cosine_similarity(text_vectors)

    similarity_between_texts =  cosine_sim_matrix

    return similarity_between_texts[0][1]

In [ ]:
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
datas['clean_text'] = datas['title'] + ' ' + datas['specifications']

In [ ]:
datas['clean_text'][0]

"Nothing Phone 1 8GB RAM 256GB Storage Non PTA 5G Black  {'RAM': '8GB', 'Memory quantity': '', 'Internal storage space': '256GB', 'Main Camera Pixels': '50 MP, f/1.9, 24mm (wide), 1/1.56', 'Battery capacity': 'Li-Po 4500 mAh, non-removable', 'Screen size': '6.55 Inches', '5G Support': 'Yes', 'Finger Print': 'Yes', 'Display technology': 'OLED, 1B colors, 120Hz, HDR10+, 500 nits (typ), 700 nits (peak)', 'Display': '6.55 inches OLED, 1B colors', 'Number of colours': '1B', 'Scratch resistant display': '', 'Screen resolution': '1080 x 2400 pixels', 'Pixel density': '402 PPI', 'Dual screens': '', 'SD Card': 'Yes', 'SDIO': '', 'Compatible memory cards': '128GB 8GB RAM, 256GB 8GB RAM, 256GB 12GB RAM', 'Maximum amount of memory': '', 'Built-in camera': 'Yes', 'Auto focus': 'Yes', 'Built-in flash': 'Yes', 'Digital zoom (still image)': 'Yes', 'Maximum resolution (still)': '1080p 30fps', 'Number of cameras': '3', 'Front Camera Resolution': '16 MP, f/2.5, (wide), 1/3.1', 'Optical zoom': 'Yes', 'Vid

In [ ]:
def get_recommendations(search_query):
  processed_query = tokenization(search_query)

  def calculate_similarity(product_text):
    return cosine_similarity_matrix(processed_query, product_text)

  datas['similarity_scores'] = datas['title'].apply(calculate_similarity)

  best_matches = datas.sort_values(by=['similarity_scores'], ascending=False).head(10)

  data = []
  for i in best_matches.index:
    data.append(best_matches['title'][i])
    data.append(best_matches['category'][i])
    data.append(best_matches['imgs'][i])

  return data

In [ ]:
# def get_recommendations(name, cosine_sim=cosine_sim):

#   recommended =[]

#   idx= datas.index[datas['title'] == name].tolist()[0]
#   sim_scores = list(enumerate(cosine_sim[idx]))
#   sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#   sim_scores = sim_scores[1:11]

#   for i in sim_scores:
#     recommended.append(datas['title'].iloc[i[0]])


#   return recommended



In [ ]:
# def get_recommendations(name, cosine_sim=cosine_sim):
#     # Cek apakah judul ada
#     matches = datas.index[datas['title'] == name].tolist()
#     if len(matches) == 0:
#         return f"Judul '{name}' tidak ditemukan di dataset."

#     idx = matches[0]

#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:11]

#     recommended = [datas['title'].iloc[i[0]] for i in sim_scores]
#     return recommended

In [ ]:
datas['title'][0]

'Nothing Phone 1 8GB RAM 256GB Storage Non PTA 5G Black '

In [ ]:
get_recommendations('Nothing Phone 1 8GB RAM 256GB Storage Non PTA 5G Black')

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['Nothing Phone 1 8GB RAM 256GB Storage Non PTA 5G Black ',
 'Mobile',
 "['https://www.mega.pk/items_images/Nothing+Phone+1+8GB+RAM+256GB+Storage+Non+PTA+5G+Black+Price+in+Pakistan%2C+Specifications%2C+Features_-_23522.webp']",
 'Lenovo ThinkPad E15 G4 Core i5 12th Generation 8GB RAM 256GB SSD DOS ',
 'Laptop',
 "['https://www.mega.pk/items_images/Lenovo+ThinkPad+E15+G4+Core+i5+12th+Generation+8GB+RAM+256GB+SSD+DOS+Price+in+Pakistan%2C+Specifications%2C+Features_-_24196.webp']",
 'Lenovo ThinkPad E15 G4 Core i5 12th Generation 8GB RAM 256GB SSD DOS ',
 'Laptop',
 "['https://www.mega.pk/items_images/Lenovo+ThinkPad+E15+G4+Core+i5+12th+Generation+8GB+RAM+256GB+SSD+DOS+Price+in+Pakistan%2C+Specifications%2C+Features_-_23711.webp']",
 'Lenovo ThinkPad E15 G4 Core i5 12th Generation 8GB RAM 512GB SSD DOS ',
 'Laptop',
 "['https://www.mega.pk/items_images/Lenovo+ThinkPad+E15+G4+Core+i5+12th+Generation+8GB+RAM+512GB+SSD+DOS+Price+in+Pakistan%2C+Specifications%2C+Features_-_23897.webp']",
 'On

In [ ]:
import pickle

with open("person.pkl", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# import pickle

# # Data to be pickled
# data = {"name": "Alice", "age": 30, "city": "New York"}

# # Pickling to a file
# with open("person.pkl", "wb") as f:
#     pickle.dump(data, f)

# # Unpickling from a file
# with open("person.pkl", "rb") as f:
#     loaded_data = pickle.load(f)

# print(loaded_data)

# # Pickling to a bytes object
# bytes_data = pickle.dumps(data)
# print(bytes_data)

# # Unpickling from a bytes object
# unpickled_data = pickle.loads(bytes_data)
# print(unpickled_data)